In [22]:
import warnings
warnings.filterwarnings('ignore')
import trimesh
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
from pathlib import Path
import pythreejs as p3s
import meshplot as mp
from bayes3d._mkl.pose import pack_pose, lift_pose, unpack_pose, look_at, from_euler

In [23]:
#|export
def create_cuboid(width=1., height=1., depth=1.):
    vertices = [
        [0, 0, 0],               # Vertex 0 (bottom-front-left)
        [width, 0, 0],           # Vertex 1 (bottom-front-right)
        [width, height, 0],      # Vertex 2 (top-front-right)
        [0, height, 0],          # Vertex 3 (top-front-left)
        [0, 0, depth],           # Vertex 4 (bottom-back-left)
        [width, 0, depth],       # Vertex 5 (bottom-back-right)
        [width, height, depth],  # Vertex 6 (top-back-right)
        [0, height, depth]       # Vertex 7 (top-back-left)
    ]
    faces = [
        [0, 1, 2, 3],  # Front face
        [1, 5, 6, 2],  # Right face
        [5, 4, 7, 6],  # Back face
        [4, 0, 3, 7],  # Left face
        [3, 2, 6, 7],  # Top face
        [4, 5, 1, 0]   # Bottom face
    ]

    cuboid_mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
    cuboid_mesh.fix_normals()

    return cuboid_mesh.vertices, cuboid_mesh.faces


def create_box(width=1., height=1., depth=1.):
    b = trimesh.primitives.Box(extents=(width, height, depth), transform=None, bounds=None, mutable=True)
    return b.vertices, b.faces

def create_cube(size=1.):
    return create_box(size, size, size)
    # return create_cuboid(size, size, size)


def create_sphere(radius=1.0, subdivisions=0):
    s = trimesh.primitives.Sphere(radius=radius, center=np.zeros(3), subdivisions=subdivisions)
    return s.vertices, s.faces

In [24]:
v,f = create_sphere(0.3, subdivisions=1)
v.shape, f.shape

((42, 3), (80, 3))

In [ ]:
vs = []
fs = []
cs = []
coloring = "VertexColors"

for _ in range(1_000):
    v,f = create_cube(.1)
    # v,f = create_sphere(0.1, subdivisions=1)
    x = np.random.randn(3)
    fs.extend(f + len(vs))
    vs.extend(v + x)

    c = np.random.rand(3)
    
    colors = np.ones_like(v)
    colors[:, 0] = c[0]
    colors[:, 1] = c[1]
    colors[:, 2] = c[2]
    cs.append(colors)

vs = np.stack(vs)
fs = np.stack(fs)
cs = np.concatenate(cs, axis=0)

print(vs.shape, fs.shape, cs.shape)

(8000, 3) (12000, 3) (8000, 3)


In [ ]:
ba_dict = {}
ba_dict["index"]    = p3s.BufferAttribute(fs.astype("uint32").ravel(), normalized=False)
ba_dict["position"] = p3s.BufferAttribute(vs, normalized=False)
ba_dict["color"]    = p3s.BufferAttribute(cs)

geometry = p3s.BufferGeometry(attributes=ba_dict)

sh = {
    "flat":True, "wireframe":False, "wire_width": 0.03, "wire_color": "black",
    "side": 'DoubleSide', "colormap": "viridis", "normalize": [None, None],
    "bbox": False, "roughness": 0.5, "metalness": 0.25, "reflectivity": 1.0,
    "line_width": 1.0, "line_color": "black",
    "point_color": "red", "point_size": 0.01, "point_shape": "circle",
    "text_color" : "red"
}
material = p3s.MeshStandardMaterial(vertexColors="VertexColors", 
                    reflectivity=sh["reflectivity"], side=sh["side"],
                    roughness=sh["roughness"], metalness=sh["metalness"], flatShading=sh["flat"],
                    polygonOffset=True, polygonOffsetFactor= 1, polygonOffsetUnits=5)

mesh = p3s.Mesh(geometry=geometry, material=material)

In [ ]:
v = mp.Viewer({
    "width": 600, 
    "height": 400, 
    "antialias": True, 
    "background": "#ccc", 
    "fov": 40})

v._cam.position = (0,0,20)
v._cam.lookAt((0,0,0))
v._light.intensity = 0.0
v._light2.intensity = 1.5
v._scene.add(mesh)

# v._orbit.exec_three_obj_method('update')
# v._cam.exec_three_obj_method('updateProjectionMatrix')


v._renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.0, positio…

## Bayes3D.Viewer

View:

- set camera by pose
- set camera by postion and attention target
- set axis for turntable controls

Meshes:

- `add_mesh` 
- `add_boxes` 
    - target mode: i.e. move center to target position, move lower right corner to target position, etc.
- `add_spheres`
- `add_lines`
    - modes: arrays of startpoints and endpoints
- `add_tabletop`: basically a box where with target reference the center on top of the table

## Viewer

In [28]:
#|export
# How a CAMERA coords "naturally" 
# embed into mp viewer coords
MP_CAM_VIEW = np.array([
    [ 1.0,   0.0,  0.0],
    [ 0.0,   1.0,  0.0],
    [ 0.0,   0.0,  -1.0]
])

# MP canonical Camera rotation
MP_WORLD_VIEW = np.array([
    [ 1.0,  0.0,  0.0],
    [ 0.0,  0.0,  1.0],
    [ 0.0,  -1.0,  0.0]
])

In [ ]:
#|export
import meshplot as mp
import numpy as np

class Viewer(object):

    def __init__(self, 
                 width  = 600, 
                 height = 400, 
                 background = "#fff", 
                 fov = 40, 
                 cam_pos  = [0, 0, 10], 
                 cam_look = [0, 0, 0],
                 view_as="world",
                 view=None,
                ):
        
        # Coordinate transform
        # To adjust mp's turntable view.
        self.T = np.eye(3)
        if view is None:
            if view_as == "world": self.T = MP_WORLD_VIEW.T
        else:
            self.T = view.T


        self._v = mp.Viewer({
            "width": width, 
            "height": height, 
            "antialias": True, 
            "scale": 1.0, 
            "background": background, 
            "fov": fov})
        
        # hack to disable auto update
        self._v.__update_view_func = self._v.__update_view
        self._v.__update_view = lambda: None

        self._v._orbit.target = tuple(cam_look@self.T)
        self._v._cam.lookAt(cam_look@self.T)
        self._v._cam.position = tuple(cam_pos@self.T)
        self._v._orbit.exec_three_obj_method('update')
        self._v._cam.exec_three_obj_method('updateProjectionMatrix')


    def add_mesh(self, vs, fs, c=np.array([1,1,1]), **kwargs):
        kwargs_ = {"shading": {"wireframe": True}}
        kwargs_.update(kwargs)

        
        self._v.add_mesh(vs@self.T, fs, c=c, **kwargs_)

    def add_points(self, vs, c=np.array([[1,1,1]]), s=1.0):
        self._v.add_points(vs@self.T, c=c, shading={"point_size": s})

    def add_pose(self, p, colors=['red', 'green', 'blue'], width=4, scale=1.0):
        x, r = unpack_pose(p)
        x = x@self.T
        for dx, c in zip(r.T, colors):
            self._v.add_lines(x[None], (x + dx@self.T)[None], 
                         shading=dict(line_width = width, line_color = c));


        return self
        
    def _repr_mimebundle_(self, **kwargs):
        return self._v._renderer._repr_mimebundle_(**kwargs)

    def look_at(self, look):
        self._v._cam.lookAt(look@self.T)
        # self._v._orbit.target = look
        self._v._orbit.exec_three_obj_method('update')
        self._v._cam.exec_three_obj_method('updateProjectionMatrix')
        return self

    def look_from(self, pos):
        self._v._cam.position = tuple(pos@self.T)
        self._v._cam.exec_three_obj_method('updateProjectionMatrix')
        return self